In [2]:
import numpy as np
import os
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [3]:
#checking for device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
print(device)

cpu


In [5]:
transformer = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.5,0.5,0.5],
                         [0.5,0.5,0.5])
]) 

In [6]:
train_path = '/home/fasal/projects/pytorch/FirstProject/Dataset/seg_train/seg_train'
test_path = '/home/fasal/projects/pytorch/FirstProject/Dataset/seg_test/seg_test'

train_loader = DataLoader(
    torchvision.datasets.ImageFolder(train_path,transform = transformer),
    batch_size = 256,
    shuffle = True
)

test_loader = DataLoader(
    torchvision.datasets.ImageFolder(test_path,transform = transformer),
    batch_size = 256,
    shuffle = True
)

In [8]:
root = pathlib.Path(train_path)
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])
print(classes)

['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


In [28]:
#cnn network
class ConvNet(nn.Module):
    def __init__(self,num_classes = 6,**kwargs):
        super(ConvNet,self).__init__()
        
        #Output size after convolution filter
        #((w-f+2P)/s)+1
        
        #Input shape(256,3,150,150)
        
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #Shape(256,12,150,150)
        self.bn1=nn.BatchNorm2d(num_features=12)
        #Shape(256,12,150,150)
        self.relu1=nn.ReLU()
        #Shape(256,12,150,150), TO BRING THE NON-LINIEARITY
        self.pool=nn.MaxPool2d(kernel_size=2)
        #Shape(256,12,75,75), TO REDUCE THE HIGHT AND WIDTH
        
        self.conv2 = nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Shape(256,20,75,75)
        self.relu2=nn.ReLU()
        #Shape(256,20,75,75)
        
        self.conv3 = nn.Conv2d(in_channels=20,out_channels=32,kernel_size=3,stride=1,padding=1)
        #Shape(256,32,75,75)
        self.bn2=nn.BatchNorm2d(num_features=32)
        #Shape(256,32,75,75)
        self.relu3=nn.ReLU()
        #Shape(256,32,75,75)
        
        self.fc=nn.Linear(in_features = 32*75*75,out_features = num_classes)
        
        
        #Feed forward functions
        
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.pool(output)
            
        output=self.conv2(output)
        output=self.relu2(output)
            
        output=self.conv3(output)
        output=self.bn2(output)
        output=self.relu3(output)
            
        #Above output will be in matrix form, with shape(256,32,75,75)
            
        output=output.view(-1,32*75*75) # IN ORDER TO FEED IT INTO A FULLY CONNECTED LAYER, RESHAPE IT
            
        output=self.fc(output)
            
        return output
        

In [29]:
model=ConvNet(num_classes=6).to(device) #CALL THE COVNET CLASS AND THEN SEND TO TO THE DEVICE

In [30]:
#Optimizer and loss function
optimizer=Adam(model.parameters(),lr=0.001,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()

In [31]:
num_epochs=10

In [32]:
#calculating training and testing dataset count
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [33]:
print(train_count,test_count)

14034 3000


In [34]:
#Model trainig and saving best model

best_accuracy=0.0

for epoch in range(num_epochs):
    
    #Evaluation and trainig on trainig dataset
    model.train()
    train_accuracy=0.0
    train_loss=0.0
    
    for i,(images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(label.cuda())
            
        optimizer.zero_grad()
        
        outputs=model(images)
        loss=loss_function(outputs,labels)
        loss.backward()
        optimizer.step()
        
        train_loss+=loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        train_accuracy+=int(torch.sum(prediction==labels.data))
        
    train_accuracy=train_accuracy/train_count
    train_loss=train_loss/train_count
    
    #Evaluation on testing dataset
    model.eval()
    
    test_accuracy=0.0
    for i,(images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(label.cuda())
            
        outputs=model(images)
        _,prediction=torch.max(outputs.data,1)
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    print('Epoch: '+str(epoch)+' Train_Loss: '+str(train_loss)+' Training_acc: '+str(train_accuracy)+' Test_acc: '+str(test_accuracy))
    
    #Save the best model
    if test_accuracy>best_accuracy:
        torch.save(model.state_dict(),'best_checkpoint.model')
        best_accuracy=test_accuracy
    

Epoch:0Train Loss:12Tain Accuracy:0.5218754453470144Test Accuracy:0.5723333333333334
Epoch:1Train Loss:1Tain Accuracy:0.6882570899244691Test Accuracy:0.6846666666666666
Epoch:2Train Loss:1Tain Accuracy:0.754952258800057Test Accuracy:0.7096666666666667
Epoch:3Train Loss:0Tain Accuracy:0.8396038192959955Test Accuracy:0.7346666666666667
Epoch:4Train Loss:0Tain Accuracy:0.867607239561066Test Accuracy:0.7163333333333334
Epoch:5Train Loss:0Tain Accuracy:0.8899814735642012Test Accuracy:0.7443333333333333
Epoch:6Train Loss:0Tain Accuracy:0.9289582442639305Test Accuracy:0.7246666666666667
Epoch:7Train Loss:0Tain Accuracy:0.9330910645575032Test Accuracy:0.751
Epoch:8Train Loss:0Tain Accuracy:0.9610232293002707Test Accuracy:0.7643333333333333
Epoch:9Train Loss:0Tain Accuracy:0.9733504346586861Test Accuracy:0.7586666666666667
